<a href="https://colab.research.google.com/github/Diwakarmogarala/Capstone/blob/main/Phase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data = pl.read_csv("/content/drive/MyDrive/Capstone Project/merged_data_with_prices.csv")

In [6]:
import polars as pl
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


encoded = le.fit_transform(data["Age Group"].to_numpy())
data = data.with_columns(
    pl.Series("Age_Group", encoded, dtype=pl.Int32)
)


In [7]:
data.columns

['Hospital County',
 'Age Group',
 'Gender',
 'Race',
 'Ethnicity',
 'Type of Admission',
 'Patient Disposition',
 'APR DRG Description',
 'APR MDC Description',
 'APR Severity of Illness Description',
 'APR Risk of Mortality',
 'APR Medical Surgical Description',
 'Payment Typology 1',
 'Payment Typology 2',
 'Payment Typology 3',
 'Length of Stay',
 'Age_Group',
 'Total Charges',
 'Total Costs',
 'Age',
 'Year']

In [8]:
data["Age_Group"].value_counts()

Age_Group,count
i32,u32
1,459545
4,1255005
2,867384
0,655299
3,1258941


In [9]:
data.head()

Hospital County,Age Group,Gender,Race,Ethnicity,Type of Admission,Patient Disposition,APR DRG Description,APR MDC Description,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Length of Stay,Age_Group,Total Charges,Total Costs,Age,Year
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i32,f64,f64,i64,i64
"""Allegany""","""70 or Older""","""F""","""White""","""Not Span/Hispanic""","""Urgent""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Moderate""","""Moderate""","""Medical""","""Medicare""","""Private Health Insurance""","""None""",3,4,3913.23,3466.83,4,2016
"""Allegany""","""30 to 49""","""M""","""White""","""Not Span/Hispanic""","""Elective""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Minor""","""Minor""","""Medical""","""Self-Pay""","""None""","""None""",2,2,3597.1,3746.8,2,2016
"""Allegany""","""50 to 69""","""F""","""White""","""Not Span/Hispanic""","""Urgent""","""Home or Self Care""","""chronic obstructive pulmonary …","""diseases and disorders of the …","""Minor""","""Minor""","""Medical""","""Medicare""","""Federal/State/Local/VA""","""None""",7,3,9149.22,8322.1,3,2016
"""Allegany""","""70 or Older""","""M""","""White""","""Not Span/Hispanic""","""Urgent""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Moderate""","""Moderate""","""Medical""","""Medicare""","""Medicaid""","""None""",4,4,5880.61,5828.87,4,2016
"""Allegany""","""50 to 69""","""M""","""White""","""Not Span/Hispanic""","""Elective""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Moderate""","""Moderate""","""Medical""","""Blue Cross/Blue Shield""","""None""","""None""",5,3,7019.76,6604.0,3,2016


In [10]:
np.corrcoef(
        data["Total Charges"].to_numpy(),
        data["Age_Group"].to_numpy()
    )[0, 1]

np.float64(0.12601413250641505)

In [11]:
data.columns

['Hospital County',
 'Age Group',
 'Gender',
 'Race',
 'Ethnicity',
 'Type of Admission',
 'Patient Disposition',
 'APR DRG Description',
 'APR MDC Description',
 'APR Severity of Illness Description',
 'APR Risk of Mortality',
 'APR Medical Surgical Description',
 'Payment Typology 1',
 'Payment Typology 2',
 'Payment Typology 3',
 'Length of Stay',
 'Age_Group',
 'Total Charges',
 'Total Costs',
 'Age',
 'Year']

In [13]:
X = data[['Hospital County',
          'Year',
          'Age Group',
          'Gender',
          'Race',
          'Ethnicity',
          'Type of Admission',
          'Patient Disposition',
          'APR DRG Description',
          'APR MDC Description',
          'APR Severity of Illness Description',
          'APR Risk of Mortality',
          'APR Medical Surgical Description',
          'Payment Typology 1',
          'Payment Typology 2',
          'Payment Typology 3'
          ]]
y = data["Length of Stay"]

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score

In [25]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

ValueError: could not convert string to float: 'Nassau'

In [26]:
from transformers import AutoTokenizer

MODEL_NAME = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]